In [22]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`
┌ Info: Precompiling BpAlignGpu [5a3eb610-29b2-4cbe-8ba2-ea97f65fa95d]
└ @ Base loading.jl:1423
  ** incremental compilation may be fatally broken for this module **



In [2]:
(q, N, L) = (21, 5, 5) 

(21, 5, 5)

In [3]:
function convert_T3_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, L)
    for i=1:L
        for n=1:N+2
            SCE_ar[n, i] = tmp[n,1,i]
            SCE_ar[N+2+n, i] = tmp[n,2,i]
        end
    end
    return SCE_ar
end
function convert_T5_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L)
    for i=1:L
        for n1=1:N+2
            for n2=1:N+2
                SCE_ar[n1, n2, i] = tmp[n1, 1, n2, 1, i]
                SCE_ar[n1, N+2+n2, i] = tmp[n1, 1, n2, 2, i]
                SCE_ar[N+2+n1, n2, i] = tmp[n1, 2, n2, 1, i]
                SCE_ar[N+2+n1, N+2+n2, i] = tmp[n1, 2, n2, 2, i]
            end
        end
    end
    return SCE_ar
end
function convert_T6_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L, L)
    for i=1:L
        for j=1:L
            for n1=1:N+2
                for n2=1:N+2
                    SCE_ar[n1, n2, i, j] = tmp[n1, 1, n2, 1, i, j]
                    SCE_ar[n1, N+2+n2, i, j] = tmp[n1, 1, n2, 2, i, j]
                    SCE_ar[N+2+n1, n2, i, j] = tmp[n1, 2, n2, 1, i, j]
                    SCE_ar[N+2+n1, N+2+n2, i, j] = tmp[n1, 2, n2, 2, i, j]
                end
            end
        end
    end
    return SCE_ar
end

convert_T6_xnTOy (generic function with 1 method)

In [4]:
T = Float32

Float32

In [5]:
using Random
header = "myseq"
myseq = randstring('A':'Z', N)
ctype=Symbol("amino")
seq = Seq(header, myseq, ctype)

LoadError: UndefVarError: Seq not defined

In [6]:
muint = 1.5
muext = 1.2
lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [7]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

LoadError: UndefVarError: ParamModel not defined

In [8]:
damp=T(0.0)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

LoadError: UndefVarError: ParamAlgo not defined

In [9]:
bpm = BPMessages(seq, pm)

LoadError: UndefVarError: BPMessages not defined

In [10]:
bpb = BPBeliefs(N, L)

LoadError: UndefVarError: BPBeliefs not defined

In [11]:
lrf = LongRangeFields(N, L)

LoadError: UndefVarError: LongRangeFields not defined

In [12]:
af = AllFields(bpm, bpb, lrf)

LoadError: UndefVarError: AllFields not defined

In [13]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);

LoadError: UndefVarError: af not defined

In [14]:
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);

LoadError: UndefVarError: af not defined

In [15]:
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

LoadError: UndefVarError: af not defined

In [16]:
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);

LoadError: UndefVarError: af not defined

In [17]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

LoadError: UndefVarError: af not defined

In [18]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

LoadError: UndefVarError: af not defined

In [19]:
tmp_Jseq = Array(af.bpm.Jseq)
elts_Jseq = convert_T6_xnTOy(tmp_Jseq);

LoadError: UndefVarError: af not defined

In [20]:
tmp_Hseq = Array(af.bpm.Hseq)
elts_Hseq = convert_T3_xnTOy(tmp_Hseq);

LoadError: UndefVarError: af not defined

In [21]:
damp = 0.0
beta = 1.0
tolnorm = 1e-5

1.0e-5

In [193]:
include("../src/SCE_update_messages.jl")
include("../src/SCE_update_shortrange_beliefs.jl")
include("../src/SCE_update_longrange.jl")

SCE_update_conditional_lr! (generic function with 1 method)

In [194]:
BpAlignGpu.update_jointchain!(af, pm)

In [195]:
i=3
res = SCE_update_joint_chain!(i, beta, tolnorm, SCE_F, SCE_B, SCE_g, lambda_o, lambda_e, N, elts_Jseq);

In [196]:
af.bpb.joint_chain[:,2,:,2,i]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0        0.0         0.0         0.0         0.0
 0.0  0.0  0.0421372  0.00463375  0.00496068  0.00286961  0.0
 0.0  0.0  0.0        0.10341     0.0209108   0.0253528   0.0
 0.0  0.0  0.0        0.0         0.0223018   0.0227328   0.0
 0.0  0.0  0.0        0.0         0.0         0.108387    0.0
 0.0  0.0  0.0        0.0         0.0         0.0         0.0
 0.0  0.0  0.0        0.0         0.0         0.0         0.0

In [197]:
rx1 = 1:N+2
rx2 = N+3:2N+4
res[rx2,rx2]

7×7 Matrix{Float64}:
 0.0  0.0  0.0        0.0         0.0         0.0         0.0
 0.0  0.0  0.0421372  0.00463375  0.00496068  0.00286961  0.0
 0.0  0.0  0.0        0.10341     0.0209108   0.0253528   0.0
 0.0  0.0  0.0        0.0         0.0223018   0.0227328   0.0
 0.0  0.0  0.0        0.0         0.0         0.108387    0.0
 0.0  0.0  0.0        0.0         0.0         0.0         0.0
 0.0  0.0  0.0        0.0         0.0         0.0         0.0

In [198]:
BpAlignGpu.update_conditional_chain!(af, pa)

In [199]:
tmp_joint = Array(af.bpb.joint_chain)
SCE_joint = convert_T5_xnTOy(tmp_joint)
tmp_cond = Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [200]:
for i=1:L-1
    SCE_update_conditional_chain!(i, SCE_cond, SCE_joint, tolnorm, N)
end

In [201]:
af.bpb.conditional[:,2,:,2,3,2]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0          0.0         0.0        0.0       0.0  0.0
 0.0  0.0          0.0         0.0        0.0       0.0  0.0
 0.0  0.567517     0.0         0.0        0.0       0.0  0.0
 0.0  0.0539384    0.367168    0.0        0.0       0.0  0.0
 0.0  0.000481443  0.00386868  0.0126095  0.0       0.0  0.0
 0.0  0.00118599   0.0033202   0.0244782  0.045385  0.0  0.0
 0.0  0.0          0.0         0.0        0.0       0.0  0.0

In [202]:
SCE_cond[rx2, rx2, 3, 2]

7×7 Matrix{Float64}:
 0.0  0.0          0.0         0.0        0.0       0.0  0.0
 0.0  0.0          0.0         0.0        0.0       0.0  0.0
 0.0  0.567517     0.0         0.0        0.0       0.0  0.0
 0.0  0.0539384    0.367168    0.0        0.0       0.0  0.0
 0.0  0.000481443  0.00386868  0.0126095  0.0       0.0  0.0
 0.0  0.00118599   0.0033202   0.0244782  0.045385  0.0  0.0
 0.0  0.0          0.0         0.0        0.0       0.0  0.0

In [203]:
SCE_update_conditional_lr!(SCE_cond)

In [235]:
SCE_cond[rx1, rx2, 2, 5]

7×7 Matrix{Float64}:
 0.0  1.0  0.875715   0.688991   0.431386    0.565488    0.0
 0.0  0.0  0.0673381  0.133579   0.0487313   0.0371608   0.0
 0.0  0.0  0.0        0.0045909  0.00947084  0.0145206   0.0
 0.0  0.0  0.0        0.0        0.0152213   0.0117331   0.0
 0.0  0.0  0.0        0.0        0.0         0.00491854  0.0
 0.0  0.0  0.0        0.0        0.0         0.0         0.0
 0.0  0.0  0.0        0.0        0.0         0.0         0.0

In [221]:
CR, CL = BpAlignGpu.update_conditional_all!(af, pm);

In [238]:
CR[:,1,:,2,2,5]

7×7 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [94]:
#BpAlignGpu.update_hB!(af, pm, pa)

In [95]:
#i=1
#af.bpm.hB[:,:,i]

In [96]:
#SCE_update_F!(i, damp, beta, tolnorm, SCE_hF, SCE_f, muext, muint, N, elts_Hseq)
#SCE_update_B!(i, damp, beta, tolnorm, SCE_hB, SCE_f, muext, muint, N, L, elts_Hseq)
#SCE_update_hF!(i, SCE_F, damp, beta, tolnorm, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)
#SCE_update_hB!(i, damp, beta, tolnorm, SCE_B, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)

In [224]:
using CUDA
x = ones(T, L, L) |> cu

5×5 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0

In [225]:
xt = ones(T, L, L)

5×5 Matrix{Float32}:
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0

In [229]:
using Tullio
@tullio x[i, j] = (i<=L-2 && j>=i+2) ? (x[i,j-1] + x[j-1,j]) : x[i,j]
@tullio x[i, j] = (j<=L-2 && i>=j+2) ? (x[i,i-1] + x[i-1,j]) : x[i,j]


5×5 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  2.0  3.0  4.0
 1.0  1.0  1.0  2.0  3.0
 2.0  1.0  1.0  1.0  2.0
 3.0  2.0  1.0  1.0  1.0
 4.0  3.0  2.0  1.0  1.0

In [232]:
for i=1:L-2
    for j=i+2:L
        xt[i,j] = xt[i,j-1]+xt[j-1,j]
    end
end    
for j=1:L-2
    for i=j+2:L
        xt[i,j] = xt[i,i-1]+xt[i-1,j]
    end
end
xt

5×5 Matrix{Float32}:
 1.0  1.0  2.0  3.0  4.0
 1.0  1.0  1.0  2.0  3.0
 2.0  1.0  1.0  1.0  2.0
 3.0  2.0  1.0  1.0  1.0
 4.0  3.0  2.0  1.0  1.0